In [1]:
import import_ipynb
import math

from tools.ToolBox import *
from tools.Midline_Edge_Reformater import *
from tools.Extract_Experiment_Characteristic import *

importing Jupyter notebook from ToolBox.ipynb
importing Jupyter notebook from Midline_Edge_Reformater.ipynb


In [2]:
def distance_point_segment(point_a, point_b, point_c):
    """
    A function that give both lateral and longitudinal absolute distance between a point and a segment/line and the smallest distance between a point and a segment.
    Note: the longitudinale distance can be negative if the projection of the point on the line bc is located 'before' the segment bc.
    In experimental condition, this might happen when midline do not fit perfectly the worm a the "start" (= tail)
    
    The Function return:
        -> seg_dist: The minimal distance between a point and a segment bc
        -> lateral_dist: the distance between the point a and its projection on the line bc 
        -> longitudinal: the distance between the point b (considered here as the "start" of the segment) and the projection of point a
        
    Arguments:
        -> point_a: X & Y coordinates of the point of interest
        -> point_b: X & Y coordinates of a segment
        -> point_c: X & Y cooridnates of a segment"""
    
    slope_origin = get_segment_slope_and_origin((point_b, point_c))
    proj_a_on_bc = orthogonal_projection(point_a, slope_origin[0][0], slope_origin[0][1])
    
    
    proj_a_to_b = segment_length(proj_a_on_bc, point_b)
    proj_a_to_c = segment_length(proj_a_on_bc, point_b)
    b_to_c = segment_length(point_b, point_c)
    
    
    if (proj_a_to_b + proj_a_to_c) > b_to_c:
        if proj_a_to_c > proj_a_to_b:
            seg_dist = segment_length(point_a, point_b)
            longitudinal_dist = - segment_length(point_b, proj_a_on_bc)
        else: 
            seg_dist = segment_length(point_a, point_c)
            longitudinal_dist = segment_length(point_b, proj_a_on_bc)
         
    else: 
        seg_dist = segment_length(point_a, proj_a_on_bc)
        longitudinal_dist = segment_length(point_b, proj_a_on_bc)
        
    lateral_dist = segment_length(point_a, proj_a_on_bc)
    
    
    if proj_a_on_bc[1] > point_a[1]:
        centr_side = 'right'
    else:
        centr_side = 'left'
    
    
    return seg_dist, lateral_dist, longitudinal_dist, centr_side

In [3]:
# This function need to accept 

def centriole_characterizator(a_centriole_coord, a_segment_list):
    """Function that compute the closest distance between a point and multiple segments and keep the smallest one
    Arguments:
        -> a_centriole_coord: X & Y coordinates of a point
        -> a_segment_list: a list of slope and origin"""
    
    index = 0
    dist = 99999999999999999999999
    
    # find the segment closest to a point + return some characteristics
    # Take care, here logitudinal_distance is the distance on the sub segment
    for segment in a_segment_list:
        new_dist, new_lateral_dist, new_long_dist, new_centr_side = distance_point_segment((a_centriole_coord[0]), segment[1], segment[2])
        
        if new_dist <= dist:
            seg_index = segment[0]
            dist, lateral_dist, longitudinal_dist, centr_side = new_dist, new_lateral_dist, new_long_dist, new_centr_side 
    
    # COMPUTE THE RELATIVE LATERAL DISTANCE
    # get the relative distance (longitudinal distance - total segment distance)
    sub_seg_dist = longitudinal_dist - a_segment_list[seg_index][4]
    
    # create length list for the appropriate [seg_index] sub_segment [-1]
    sub_list = []
    [sub_list.append(ele[0]) for ele in a_segment_list[seg_index][-1]]
    

    # search in the list the closest to sub_seg_dist
    idx_sub_seg = min(range(len(sub_list)), key=lambda x:abs(sub_list[x]-sub_seg_dist))

    if a_segment_list[seg_index][-1][idx_sub_seg][1][1][0] == centr_side:
        lateral_rel = lateral_dist/(a_segment_list[seg_index][-1][idx_sub_seg][1][1][1])
    
    else: 
        lateral_rel = lateral_dist/(a_segment_list[seg_index][-1][idx_sub_seg][1][0][1])

    
    # COMPUTE RELATIVE LONGITUDINAL DISTANCE
    worm_length = a_segment_list[-1][4] + a_segment_list[-1][3]
    
    longitudinal_dist = longitudinal_dist+ a_segment_list[seg_index][4]
    longitudinal_rel = longitudinal_dist/worm_length
    
    # COMPUTE NEW ANGLE
    newAngle = a_centriole_coord[1] - a_segment_list[seg_index][5]
    newAngle = math.degrees(math.atan2(math.sin(math.radians(newAngle)), math.cos(math.radians(newAngle))))    
    
    return seg_index, lateral_dist, longitudinal_dist, centr_side, lateral_rel, longitudinal_rel, newAngle

In [ ]:
def centriole_shifter(centrioleList, pathExcel = None, Xls_extracted = None, problem = 'classification'):
    """ Function that shif the X/Y cordinates of centriole according to a user pre-define shifting value
    The function accept either a path to an Xls file or the extracted data of an xls file"""
    
    if pathExcel != None:
        db = get_xls_values(pathExcel)
    
    elif Xls_extracted != None:
        db = Xls_extracted
        
    else: 
        raise 'No data provided'
    
    # Extract the shift
    shiftX = db['image_shift']['x']
    shiftY = db['image_shift']['y']
    
    # If no shift => no modification to do
    if shiftX == 0 and shiftY == 0:
        return centrioleList
    
    else:
        shifted_centriole_list = []

        for a_centriole in a_list_of_centriole:
            xShifted = a_centriole[0][0] + shiftX
            yShifted = a_centriole[0][1] + shiftY
            
            if problem == 'classification':
                shifted_centriole_list.append(((xShifted, -yShifted),a_centriole[1]*5+2.5))
            else:
                shifted_centriole_list.append(((xShifted, -yShifted),a_centriole[1]))
    
        return shifted_centriole_list 

In [ ]:
def centriole_compensation(centrioleList, midline):
    reoriented_centriole = []
    for a_centriole in centrioleList:
        tmp_list = list(centriole_characterizator(a_centriole, midline))
        if midline[4] == 'gauche' or midline[4] == 'left':
            tmp_list[-2] = 1 - tmp_list[-2]
            tmp_list[-1] = math.degrees(math.atan2(-math.sin(math.radians(tmp_list[-1])), -math.cos(math.radians(tmp_list[-1]))))
        tmp_list.insert(1,a_centriole[0][0])
        tmp_list.insert(2,a_centriole[0][1])

        reoriented_centriole.append(tmp_list)

    return reoriented_centriole 